In [1]:
import psycopg2

In [2]:
conn = psycopg2.connect(database="first", user="postgres", password="774165", host="127.0.0.1", port="5432")

In [1]:
import pandas as pd
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine

In [3]:
def import_csv_to_pg_use_df(database, user, password, host, port, csv_file_path, pg_table_name):
    connect = create_engine(
    f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')
    # 如果有bool类型的数据，那么可以测试一下，可以使用sqlalchemy.types.VARCHAR或者sqlalchemy.types.Boolean
    dtype_dict = {
    "haha_id": sqlalchemy.types.VARCHAR,
    "finch_xu_text": sqlalchemy.types.TEXT,
    "haha_score": sqlalchemy.types.Float,
    "haha_age": sqlalchemy.types.Integer,
    "haha_birthday": sqlalchemy.types.TIMESTAMP
    }
    try:
        # 由于导出时候设置双引号，那么这里读进来就是认为都是字符串。所以这里设置quoting=1和dtype=str
        df = pd.read_csv(csv_file_path, encoding='utf-8', quoting=1, dtype=str)
        # 这里写入数据库，我们设置了字典以保证字段类型和数据库的类型匹配。
        df.to_sql(name=pg_table_name,
        con=connect,
        index=False,
        if_exists='append',
        dtype=dtype_dict)
    except Exception as e:
        print(str(e))

In [4]:
import_csv_to_pg_use_df(database="first", user="postgres", password="774165", host="127.0.0.1", port="5432", csv_file_path='../export_csv/data/edge_list.csv', pg_table_name='edge_list')

In [2]:
df = pd.DataFrame()
sql_query = "select * from "
def output_pg_to_csv_use_df(sql_query, pg_table_name):
    global df
    connect = psycopg2.connect(database="first", user="postgres", password="774165", host="127.0.0.1", port="5432")
    try:
        # pg_output_csvfile_name = str(file_ts) + pg_table_name + "_pg_output.csv"
        df = pd.read_sql_query(sql_query+pg_table_name, con=connect)
        # print(df_result.head())
        # 设置 quoting=1 是设置每个值都加上双引号以隔离开每个数据。
        # 既然csv无法设置字段的数据类型，那么我们加上双引号，把所有类型都设定为字符串。
        # df.to_csv(os.path.join(ouput_csvfile_path, pg_output_csvfile_name),
        # index=False,
        # header=True,
        # encoding="utf-8", 
        # quoting=1)
    except Exception as e:
        print(F'查询失败，详情:{e}')
    finally:
        connect.close()

In [3]:
output_pg_to_csv_use_df(sql_query, pg_table_name='salary')
df

,name,salary,years_experience
0,Keagan Klocko,92239.0868697,4
1,Brandy Collier,61086.9992117,2
2,Veronica Okuneva,47298.0146627,3
3,Shana Beahan,59978.9111169,3
4,Ayden Toy,81590.5826583,2
...,...,...,...
244,Emie Hansen,75140.5711447,2
245,Torrance Marquardt,49592.0750131,3
246,Myrna Barton,53852.4403867,4
247,Morris Thiel,40272.1841174,3
